In [1]:
import pandas as pd
from datetime import timedelta,datetime

In [2]:
provinces = ['BKK','Chiangmai','Rayong','Saraburi','Khonkaen','Surat']

for i in range(len(provinces)):
    province = provinces[i]

    data_test = pd.read_csv(province+'_clean2.csv', date_parser = True)
    data_test['date_time'] = pd.to_datetime(data_test['date_time'])
    
    data_pred = pd.read_csv(province+'_pred.csv')

    pred_time = []
    pm = []
    date_time = []

    for i in range(len(data_test)):
        for j in range(1,13):
            d = data_test.iloc[i,0]
            pred_time.append(d+j*timedelta(hours=6))
            date_time.append(d)
            pm.append(data_pred.iloc[i*6,j-1])

    data = pd.DataFrame({'date_time':date_time,'date_pred':pred_time,'PM2.5':pm})
    data['date_time'] = pd.to_datetime(data['date_time'])

    missing_test_date = pd.read_csv('../_missing_date/missing_test_date_'+province.lower()+'.csv')

    missing_test_date.columns = ['','date_pred']
    missing_test_date['date_pred'] = pd.to_datetime(missing_test_date['date_pred'] )

    df = pd.merge(data, missing_test_date, how='outer', indicator=True)
    df = df.loc[df._merge == 'left_only', ['date_time',	'date_pred','PM2.5']]
    df = df.sort_values(['date_time','date_pred'])
    df['province']=province

    df.to_csv(province+'_submit.csv',index=False)

In [3]:
df = pd.DataFrame({'date_time':[],'date_pred':[],'PM2.5':[],'province':[]})
for province in provinces:
    data = pd.read_csv(province+'_submit.csv')
    print(province,data.shape)
    df = pd.concat([df,data])
df.to_csv('all_pred.csv',index=False)
df.head()

BKK (15588, 4)
Chiangmai (15768, 4)
Rayong (16020, 4)
Saraburi (15564, 4)
Khonkaen (15648, 4)
Surat (16128, 4)


,date_time,date_pred,PM2.5,province
0,2020-07-01 00:00:00,2020-07-01 06:00:00,12.321317,BKK
1,2020-07-01 00:00:00,2020-07-01 12:00:00,12.993484,BKK
2,2020-07-01 00:00:00,2020-07-01 18:00:00,11.675415,BKK
3,2020-07-01 00:00:00,2020-07-02 00:00:00,10.370315,BKK
4,2020-07-01 00:00:00,2020-07-02 06:00:00,13.185866,BKK


In [4]:
all_pred_clean = pd.read_csv('all_pred_clean.csv')
all_pred_clean.columns = ['date_time','date_pred','PM2.5','province']
all_pred = pd.read_csv('all_pred.csv')

In [5]:
m = all_pred_clean.merge(all_pred,how='right',on=['date_time','date_pred','province'])
m.dropna(inplace=True)
m.reset_index(inplace=True)

In [6]:
import math
from sklearn.metrics import mean_squared_error
for province in provinces:
    print( province,math.sqrt(mean_squared_error(m[m['province']==province]['PM2.5_x'], m[m['province']==province]['PM2.5_y'])) )



print(math.sqrt(mean_squared_error(m['PM2.5_x'], m['PM2.5_y'])) )

BKK 8.063850622272051
Chiangmai 11.121885785256602
Rayong 7.708040873047741
Saraburi 11.273307578448222
Khonkaen 11.047862056258944
Surat 6.49011554171544
9.463725485542943
